# 🚀 **LIGHT MODEL : pipeline complet**

Pipeline **content-based** simple, rapide et optimisé pour le CPU.

**Caractéristiques :**
- Léger et reproductible
- Pas de dépendances GPU
- Idéal pour prototypage ou petits datasets

**Étapes :**
1. Charge la configuration YAML
2. Charge le dataset (`anime_master_clean.csv`)
3. Construit les vecteurs de features :
   - TF-IDF sur les synopsis (10k tokens max)
   - One-hot encoding des genres
   - Normalisation des features numériques
4. Concatène les features en une matrice unique
5. Construit un modèle KNN exact (`sklearn`)
6. Sauvegarde :
   - Matrice des features (`features.npz`)
   - Métadonnées (`metadata.json`)
   - Modèle KNN (`knn_model.pkl`)
   - Configuration utilisée (`config_used.yaml`)
   - Logs d'exécution (`training.log`)
7. Optionnel : auto-benchmark (précision/latence)

**Sorties :**

project/
├── features.npz
├── metadata.json
├── knn_model.pkl
├── config_used.yaml
└── training.log


**Avantages :**
✅ Rapide à entraîner (<1 min pour 30k animes)
✅ Faible consommation mémoire
✅ Pas de dépendances complexes

**Cas d'usage :**
- Prototypage rapide
- Environnements sans GPU
- Tests de base avant passage au modèle Premium


In [34]:
# ============================================================
# Notebook : 03_train_content_light
# Objectif : Entraînement du modèle Content-Based LIGHT v2
# Pipeline : TF-IDF + Genres + Numerical → KNN
# ============================================================

import os
import json
import yaml
import joblib
import pandas as pd
import numpy as np

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import MinMaxScaler
from sklearn.neighbors import NearestNeighbors
from scipy import sparse

from datetime import datetime

In [35]:
# ============================================================
# Chargement configuration
# ============================================================

ROOT = os.path.abspath(os.path.join(os.getcwd(), ".."))

config_path = os.path.join(ROOT, "anime-recommender", "config", "content_light.yaml")
print("Loading:", config_path)

with open(config_path, "r") as f:
    config = yaml.safe_load(f)


Loading: c:\Users\isabe\Desktop\machineLEARNIN\PROJET\anime-recommender\config\content_light.yaml


In [36]:
# ============================================================
# Chargement du dataset
# ============================================================

df_path = config["data"].get("dataset_path")
if df_path is None:
    raise ValueError("❌ dataset_path manquant dans le YAML.")

df = pd.read_csv(df_path)
df = df.dropna(subset=["synopsis"])
print("Dataset:", df.shape)

Dataset: (23827, 23)


In [37]:
# ============================================================
# Run directory
# ============================================================

run_id = datetime.now().strftime("%Y%m%d_%H%M%S")
run_dir = f"runs/content_light/{run_id}"
os.makedirs(run_dir, exist_ok=True)


In [38]:
# ============================================================
# 1. TF-IDF
# ============================================================

tfidf_params = config["model"]["tfidf"]
vectorizer = TfidfVectorizer(
    max_features=tfidf_params["max_features"],
    ngram_range=tuple(tfidf_params["ngram_range"]),
    min_df=tfidf_params["min_df"],
    stop_words="english"
)

tfidf_matrix = vectorizer.fit_transform(df["synopsis"])
print("TF-IDF:", tfidf_matrix.shape)


TF-IDF: (23827, 20000)


In [39]:
# ============================================================
# 2. One-Hot Genres
# ============================================================

if "genres" in df.columns:
    genre_matrix = df["genres"].fillna("").str.get_dummies(sep=", ")
else:
    genre_matrix = pd.DataFrame()

if len(genre_matrix) > 0:
    genre_sparse = sparse.csr_matrix(genre_matrix.values)
    print("Genres:", genre_sparse.shape)
else:
    genre_sparse = None
    print("Aucun genre trouvé.")


Aucun genre trouvé.


In [40]:
# ============================================================
# 3. Features numériques (normalisées)
# ============================================================

num_cols = ["score", "members", "episodes", "duration"]
num_cols = [c for c in num_cols if c in df.columns]

if len(num_cols) > 0:
    scaler = MinMaxScaler()
    num_scaled = scaler.fit_transform(df[num_cols].fillna(0))
    num_sparse = sparse.csr_matrix(num_scaled)
    print("Numerical:", num_sparse.shape)
else:
    num_sparse = None
    print("Aucune feature numérique.")

Numerical: (23827, 2)


In [41]:
# ============================================================
# 4. Concaténation GLOBAL FEATURES
# ============================================================

parts = [tfidf_matrix]

if genre_sparse is not None:
    parts.append(genre_sparse)

if num_sparse is not None:
    parts.append(num_sparse)

features = sparse.hstack(parts).tocsr()

print("Final feature matrix:", features.shape)

Final feature matrix: (23827, 20002)


In [42]:
# ============================================================
# 5. KNN (brute force cosine)
# ============================================================

knn_params = config["model"]["knn"]
knn = NearestNeighbors(
    n_neighbors=knn_params["n_neighbors"],
    metric="cosine",
    algorithm="brute"
)

knn.fit(features)
print("KNN trained (cosine, brute force)")


KNN trained (cosine, brute force)


In [43]:
# ============================================================
# Sauvegarde
# ============================================================

sparse.save_npz(f"{run_dir}/features.npz", features)
df.to_csv(f"{run_dir}/training_data.csv", index=False)

joblib.dump(vectorizer, f"{run_dir}/tfidf_vectorizer.joblib")
joblib.dump(knn, f"{run_dir}/knn_model.joblib")

if len(genre_matrix) > 0:
    joblib.dump(genre_matrix.columns.tolist(), f"{run_dir}/genre_columns.joblib")

if len(num_cols) > 0:
    joblib.dump({"cols": num_cols, "scaler": scaler}, f"{run_dir}/numerical_scaler.joblib")


In [44]:
# ============================================================
# Fonction de recommandation
# ============================================================

from sklearn.metrics.pairwise import cosine_similarity

def find_best_match(title, df):
    m = df[df["title"].str.contains(title, case=False, na=False)]
    if len(m) >= 1:
        return m.index[0]
    return None


def recommend(title, df=df, features=features, knn=knn, top_k=10):
    idx = find_best_match(title, df)
    if idx is None:
        return {"error": f"Aucun anime trouvé pour : {title}"}

    print(f"🎯 Point de départ : {df.loc[idx, 'title']} (index={idx})")

    distances, indices = knn.kneighbors(features[idx], n_neighbors=top_k+1)
    inds = indices[0][1:]
    sims = 1 - distances[0][1:]

    recs = df.iloc[inds][["title"]].copy()
    recs["score"] = sims.round(3)

    return {"query": df.loc[idx, "title"], "results": recs.to_dict(orient="records")}



In [50]:
# ============================================================
# Fonction robuste de recommandation
# ============================================================

try:
    from rapidfuzz import process as rf_process
    HAS_RAPIDFUZZ = True
except Exception:
    HAS_RAPIDFUZZ = False

from sklearn.metrics.pairwise import cosine_similarity

def find_best_match(title, df):
    # 1) exact / substring
    matches = df[df["title"].str.contains(title, case=False, na=False)]
    if len(matches) == 1:
        return matches.index[0]
    if len(matches) > 1:
        return matches.sort_values("members", ascending=False).index[0]

    # 2) fuzzy fallback
    if HAS_RAPIDFUZZ:
        choices = df["title"].tolist()
        best = rf_process.extractOne(title, choices)
        if best and best[1] > 70:  # score threshold
            return df.index[df["title"] == best[0]][0]

    # 3) last resort: closest substring using simple python ratio
    lower_titles = df["title"].str.lower().tolist()
    low = title.lower()
    best_idx = None
    best_score = 0
    for i, t in enumerate(lower_titles):
        sc = sum(1 for c in low.split() if c in t)
        if sc > best_score:
            best_score = sc
            best_idx = i
    if best_idx is not None and best_score > 0:
        return df.index[best_idx]

    return None

def recommend(title, df=df, features=features, knn=knn, top_k=10):
    idx = find_best_match(title, df)
    if idx is None:
        return {"error": f"Aucun anime trouvé pour '{title}'."}

    print(f"🎯 Anime utilisé comme point de départ : {df.loc[idx, 'title']} (index={idx})")

    distances, indices = knn.kneighbors(features[idx], n_neighbors=top_k+1)

    inds = indices[0][1:]
    sims = 1 - distances[0][1:]

    recs = df.iloc[inds][["title"]].copy()
    recs["score"] = sims.round(3)

    return {
        "query": df.loc[idx, "title"],
        "results": recs.to_dict(orient="records")
    }



# 1) vérifier synopsis pour Naruto
cands = df[df["title"].str.contains("Naruto", case=False, na=False)]
print("hits:", len(cands))
print(cands[["title","synopsis"]].head(5).to_string())

# 2) voisins en TF-IDF pur (sans UMAP) pour comparer
from sklearn.metrics.pairwise import cosine_similarity
idx0 = cands.index[0]
sim = cosine_similarity(tfidf_matrix[idx0], tfidf_matrix).ravel()
top = sim.argsort()[::-1][1:11]
print("Top TF-IDF neighbors:", df.iloc[top]["title"].tolist())


out = recommend("Naruto", top_k=10)
if "error" in out: print(out["error"])
else:
    for r in out["results"]:
        print(f"{r['score']:.3f}", r["title"])



hits: 29
                                                         title                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                 

In [46]:
# ============================================================
# TEST Naruto
# ============================================================

out = recommend("Naruto", top_k=10)
print(out)

🎯 Anime utilisé comme point de départ : Naruto (index=16814)
{'query': 'Naruto', 'results': [{'title': 'Poporisu & Friends', 'score': 0.4728938907730599}, {'title': 'Ugoku Ehon', 'score': 0.4674344312948099}, {'title': '5-tsu no Yubi no Niwa', 'score': 0.41791336390393574}, {'title': 'Ezaki Glico Pocky Web CMs', 'score': 0.3977603099894541}, {'title': 'Beads no Mori no Rabi', 'score': 0.3855123516349174}, {'title': 'Maze', 'score': 0.38451105900792865}, {'title': 'Nendoroid 15-shuunen Anniversary PV', 'score': 0.3743366761448931}, {'title': 'Majin Sentai Death Ranger: Ubawareta Pudding!', 'score': 0.36396174222625977}, {'title': 'Project Kabigon', 'score': 0.3374036272549915}, {'title': 'Tong Ching Kahn', 'score': 0.33089826064683636}]}


In [47]:
print("=== TRAINING REPORT ===")
print("Rows:", df.shape)
print("TF-IDF:", tfidf_matrix.shape)
print("Final features:", features.shape)
print("Saved in:", run_dir)

=== TRAINING REPORT ===
Rows: (23827, 23)
TF-IDF: (23827, 20000)
Final features: (23827, 20002)
Saved in: runs/content_light/20251114_094811
